# Transformace PDF v Tabula

In [20]:
import os
import re
import pandas as pd
import tabula

### Zjistime, ve kterem adresari jsme

In [21]:
os.getcwd()

'D:\\nagympl\\nagympl\\jupyter_datacleaning'

In [22]:
### Seznam vseho, co je v danem adresari

In [23]:
os.listdir()

['.ipynb_checkpoints',
 'Prvotni cisteni csv.ipynb',
 'transformace PDF v Tabula a cisteni.ipynb']

### Promenne rok a skola,se kterymi pracujeme

year = 2017
schoolid = 16

In [24]:
year = 2018
schoolid = "05"

### Slozka v pocitaci, kde jsou podslozky s jednotlivymi skolami nebo kam chceme ukladat - pokud nejsme tam, lze vlozit (pripadne ulozit notebook do prislusne slozky)

In [25]:
path = (f'D:/nagympl/trans_PDF/ostre soubory/{schoolid}/')

In [26]:
print(path)

D:/nagympl/trans_PDF/ostre soubory/05/


#### seznam souboru v dane slozce

In [13]:
files = os.listdir(path)
print(files)

['05-2019-Opatov.pdf', '05-2020-Opatov.pdf', '05_2017.pdf', '05_2017_pythonapi.csv', '05_2018.pdf', '05_2018_pythonapi.csv', '05_2019_pythonapi.csv', '05_2020_pythonapi.csv']


#### vybirame si soubor

In [27]:
file = path + files[4]
print(file)
#./trans_PDF/ostre soubory/16/16-2017.csv

D:/nagympl/trans_PDF/ostre soubory/05/05_2018.pdf


### Soubor je na webu
file_web = "www.neco.nekde/x.pdf"

file = "http://www.matematickaberuska.cz/vysledky/2017/Praha/st%c3%a1tn%c3%ad/www.omska.cz//VyslPrij_08_2017.pdf"

### Nacitani souboru pomoci Tabuly
file - na pocitaci, file_web - na webu

In [29]:
table_raw = tabula.read_pdf(
    file,
    pages="all",
    multiple_tables=True,
    output_format='dataframe')

CSVParseError: Error failed to create DataFrame with different column tables.
Try to set `multiple_tables=True`or set `names` option for `pandas_options`. 
, caused by ParserError('Error tokenizing data. C error: Expected 2 fields in line 40, saw 6\n')

### Kontrola

In [ ]:
print(table_raw)
print(type(table_raw))

In [ ]:
print(len(table_raw))

In [ ]:
print(type(table_raw[0]))

In [ ]:
table = table_raw[0]


In [ ]:
print(type(table))

In [ ]:
table.info()

### Pripadne prejmenovani sloupcu a kontrola

table.rename(
    columns=
        {'poradi': 'RANK',
         'matemat.': 'MATH',
         'čeština': 'CZECH',
         'Celkem': 'SUM',
         'ZŠ': 'GRADES',
         'reg. číslo': 'ID'},
    inplace=True)
print(table)

### Hledani 'schovanych' bodu za souteze: 
pokud jsou schovane pod SUM, bude SUM vetsi nez soucet vsech bodu. Pokud jsou schovane pod GRADES, moc s tim nenadelame, ale muzeme vyfiltrovat ty, kdo ma vic bodu v GRADES nez maximalni mozny pocet, a odecist 

table.loc[soubor1[["CZECH", "MATH", "GRADES"]].sum(axis=1) != soubor1["SUM"]]
table

## Pripadna uprava RANK 
deleni slozeneho poradi, vymazani pomlcky a rozpeti, prevod na numericky datovy typ

#kontrola
print(soubor1['RANK'].head(35))

### RANK transformation - test, jestli funguje jak ma 

test_string = soubor1['RANK'][28] # test also on index 0
re.sub('[^0-9]+', '', test_string.split('-')[0])

### RANK transformation + prevod na int + kontrola

In [ ]:
soubor1['RANK'] = [re.sub('[^0-9]+', '', s.split('-')[0]) for s in soubor1['RANK']]

In [ ]:
soubor1['RANK'] = soubor1['RANK'].astype(int)

### kontrola
soubor1.tail(50)

### Vytvoreni sloupcu YEAR a SCHOOLCOURSE_IS

In [ ]:
soubor1['YEAR'] = year

In [ ]:
soubor1['SCHOOLCOURSE_ID'] = int(str(schoolid) + "81")

## Ulozeni souboru 
rok a skola do nazvu jsou doplneny automaticky

In [ ]:
soubor1.to_csv(f"{path}/{schoolid}_{year}_clean.csv", encoding='utf-8')